In [ ]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from keras.models import Sequential
from keras.layers import Dense, Activation, Input, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2

from utils import input_columns_names

In [ ]:
%matplotlib inline

In [ ]:
PICTURE_SIZE = 100
CHANNELS = 'rgb'

INPUT_COLUMNS = input_columns_names(PICTURE_SIZE)

In [ ]:
print('reading data...')
data = pd.read_pickle('./data/object_picture_sets/sword/dataframe_{}.pkl'.format(PICTURE_SIZE))
data.shape

In [ ]:
data.sample(5)

In [ ]:
def show_images(samples, title=('label', 'file')):
    for index, sample in samples.iterrows():
        if title is not None:
            if isinstance(title, str):
                title = [title, ]
            title_text = ', '.join(str(sample[title_field]) for title_field in title)
            plt.title(title_text)

        sample_as_grid = sample[INPUT_COLUMNS].values.reshape(len(CHANNELS), PICTURE_SIZE, PICTURE_SIZE).astype(np.float)
        sample_as_grid = np.transpose(sample_as_grid, (1, 2, 0)) / 255
        
        plt.axis('off')
        plt.imshow(sample_as_grid, interpolation='nearest')

        plt.show()

In [ ]:
show_images(data[data.label == 1].sample(3))

In [ ]:
show_images(data[data.label == 0].sample(3))

In [ ]:
train, test = train_test_split(data, test_size=0.2)
train = train.copy()
test = test.copy()

sets = (
    ('train', train),
    ('test', test),
)

for set_name, set_data in sets:
    set_data.label.hist()
    plt.show()

In [ ]:
def extract_inputs(dataset):
    return dataset[INPUT_COLUMNS].values.reshape(len(dataset), PICTURE_SIZE, PICTURE_SIZE, len(CHANNELS)) / 255

def extract_outputs(dataset):
    return dataset.label.values

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='tanh', kernel_regularizer=l2(0.01), input_shape=(PICTURE_SIZE, PICTURE_SIZE, len(CHANNELS))),
    Conv2D(16, (3, 3), activation='tanh', kernel_regularizer=l2(0.01)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(32, (3, 3), activation='tanh', kernel_regularizer=l2(0.01)),
    Conv2D(16, (3, 3), activation='tanh', kernel_regularizer=l2(0.01)),
    Conv2D(16, (3, 3), activation='tanh', kernel_regularizer=l2(0.01)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(32, (3, 3), activation='tanh', kernel_regularizer=l2(0.01)),
    Conv2D(16, (3, 3), activation='tanh', kernel_regularizer=l2(0.01)),
    Conv2D(16, (3, 3), activation='tanh', kernel_regularizer=l2(0.01)),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    
    Dense(1000, activation='tanh', kernel_regularizer=l2(0.01)),
    Dropout(0.2),
    
    Dense(1, activation='sigmoid'),
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy',],
)

In [ ]:
augmentator = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    
    vertical_flip=False,   
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
)
augmentator.fit(extract_inputs(train))

In [ ]:
model.fit_generator(
    augmentator.flow(
        extract_inputs(train), 
        extract_outputs(train),
        batch_size=128,
    ),
    steps_per_epoch=len(train),
    epochs=20,
)

In [ ]:
def add_predictions(dataset):
    dataset['model_output'] = model.predict(extract_inputs(dataset))
    dataset['prediction'] = np.rint(dataset.model_output.values)
    dataset['correct'] = dataset.prediction == dataset.label

In [ ]:
for set_name, set_data in sets:
    add_predictions(set_data)
    
    print('#' * 25, set_name, '#' * 25)
    print('accuracy', accuracy_score(set_data.label, set_data.prediction))
    print(classification_report(set_data.label, set_data.prediction))

In [ ]:
show_images(test[(test.correct) & (test.label == 1)].sample(3), title=['label', 'model_output'])

In [ ]:
show_images(test[(~test.correct) & (test.label == 1)].sample(3), title=['label', 'model_output'])

In [ ]:
show_images(test[(~test.correct) & (test.label == 0)].sample(3), title=['label', 'model_output'])